In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm import tqdm #for문 진행률
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_2월.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [4]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

100%|██████████| 13353/13353 [34:16<00:00,  6.49it/s]  


,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/4t1eu3f/222660518804,Spread Love Not Hate,2022.02.28.,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지!\n...,0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,[(집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 ...
1,https://blog.naver.com/olive07100/222660518370,김나륜 작품 초대전,2022.02.28.,안녕하세요 꼼꼼여사에요\n오늘은 김나륜 작품 초대전을 다녀왔어요~\n저는 그림을 잘...,1,안녕하세요 꼼꼼여사에요 오늘은 김나륜 작품 초대전을 다녀왔어요 저는 그림을 잘 모르...,"[(안녕하세요 꼼꼼여사에요, 0, 12, None, []), (오늘은 김나륜 작품 ..."
2,https://blog.naver.com/7046476/222660518937,2월의 문화생활 기록장,2022.02.28.,다큐멘터리\n미스 아메리카나\n줄거리: 테일러 스위프트의 삶\n- 유튜브에서 추천하...,2,다큐멘터리 미스 아메리카나 줄거리 테일러 스위프트의 삶 유튜브에서 추천하길래 봄 공...,[(다큐멘터리 미스 아메리카나 줄거리 테일러 스위프트의 삶 유튜브에서 추천하길래 봄...
3,https://blog.naver.com/designpress2016/2226605...,국내 건축상 휩쓴 화제의 지방 휴게소... 자연에 순응하는...,2022.02.28.,대한민국공간문화대상 대상\n황매산군립공원 휴게소 '철쭉과 억새 사이'\n관광객이 모...,3,대한민국공간문화대상 대상 황매산군립공원 휴게소 철쭉과 억새 사이 관광객이 모이는 곳...,[(대한민국공간문화대상 대상 황매산군립공원 휴게소 철쭉과 억새 사이 관광객이 모이는...
4,https://blog.naver.com/ogooodman/222660517918,서울은 도시가 아니다,2022.02.28.,#046\n★★★★★\n단상 / 건축과 도시\n파일럿\n도시는 건축의 확장형이다.\...,4,046 단상 건축과 도시 파일럿 도시는 건축의 확장형이다 적어도 도시는 그 시대의 ...,"[(046 단상 건축과 도시 파일럿 도시는 건축의 확장형이다, 0, 31, None..."
...,...,...,...,...,...,...,...
13348,https://blog.naver.com/bubi-/222636081111,신해철 와이프 윤원희 똑 닮은 자녀 + 죽음 이유 집도의 강세훈...,2022.02.01.,31일 밤 방송된 KBS 2TV 자본주의학교에는\n도(故) 신해철 와이프 윤원희 씨...,13348,31일 밤 방송된 KBS 2TV 자본주의학교에는 도 신해철 와이프 윤원희 씨 그리고...,[(31일 밤 방송된 KBS 2TV 자본주의학교에는 도 신해철 와이프 윤원희 씨 그...
13349,https://blog.naver.com/talgodang/222635787110,설날에 심심풀이 땅콩으로 보는 '2022 임인년' 올해의 운세,2022.02.01.,2022 임인년 설날입니다.\n명절 연휴 다들 잘 보내고 계신지요? 저는 차례 드릴...,13349,2022 임인년 설날입니다 명절 연휴 다들 잘 보내고 계신지요 저는 차례 드릴 준비...,[(2022 임인년 설날입니다 명절 연휴 다들 잘 보내고 계신지요 저는 차례 드릴 ...
13350,https://blog.naver.com/imbee1229/222636077004,신세계백화점 타임스퀘어점 문화센터 꽃꽂이!,2022.02.01.,신세계백화점 타임스퀘어점\n문화센터 꽃꽂이 수강 후기!\n50m\n© NAVER C...,13350,신세계백화점 타임스퀘어점 문화센터 꽃꽂이 수강 후기 50m NAVER Corp 신세...,[(신세계백화점 타임스퀘어점 문화센터 꽃꽂이 수강 후기 50m NAVER Corp ...
13351,https://blog.naver.com/kji206/222635993490,"전북 가볼만한곳 고창 고인돌유적지,운곡습지,고창읍성 등",2022.02.01.,"전북 가볼만한곳 고창 고인돌유적지,운곡습지,고창읍성 등\n고창은 서해바다 여행도 할...",13351,전북 가볼만한곳 고창 고인돌유적지 운곡습지 고창읍성 등 고창은 서해바다 여행도 할 ...,[(전북 가볼만한곳 고창 고인돌유적지 운곡습지 고창읍성 등 고창은 서해바다 여행도 ...


In [5]:
import pickle
# 데이터프레임을 피클 파일로 저장
with open('2월_문장분리.pkl', 'wb') as file:
    pickle.dump(df, file)

In [6]:
# 저장된 피클 파일을 불러오기
with open('2월_문장분리.pkl', 'rb') as file:
    loaded_df = pickle.load(file)
loaded_df

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/4t1eu3f/222660518804,Spread Love Not Hate,2022.02.28.,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지!\n...,0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,[(집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 ...
1,https://blog.naver.com/olive07100/222660518370,김나륜 작품 초대전,2022.02.28.,안녕하세요 꼼꼼여사에요\n오늘은 김나륜 작품 초대전을 다녀왔어요~\n저는 그림을 잘...,1,안녕하세요 꼼꼼여사에요 오늘은 김나륜 작품 초대전을 다녀왔어요 저는 그림을 잘 모르...,"[(안녕하세요 꼼꼼여사에요, 0, 12, None, []), (오늘은 김나륜 작품 ..."
2,https://blog.naver.com/7046476/222660518937,2월의 문화생활 기록장,2022.02.28.,다큐멘터리\n미스 아메리카나\n줄거리: 테일러 스위프트의 삶\n- 유튜브에서 추천하...,2,다큐멘터리 미스 아메리카나 줄거리 테일러 스위프트의 삶 유튜브에서 추천하길래 봄 공...,[(다큐멘터리 미스 아메리카나 줄거리 테일러 스위프트의 삶 유튜브에서 추천하길래 봄...
3,https://blog.naver.com/designpress2016/2226605...,국내 건축상 휩쓴 화제의 지방 휴게소... 자연에 순응하는...,2022.02.28.,대한민국공간문화대상 대상\n황매산군립공원 휴게소 '철쭉과 억새 사이'\n관광객이 모...,3,대한민국공간문화대상 대상 황매산군립공원 휴게소 철쭉과 억새 사이 관광객이 모이는 곳...,[(대한민국공간문화대상 대상 황매산군립공원 휴게소 철쭉과 억새 사이 관광객이 모이는...
4,https://blog.naver.com/ogooodman/222660517918,서울은 도시가 아니다,2022.02.28.,#046\n★★★★★\n단상 / 건축과 도시\n파일럿\n도시는 건축의 확장형이다.\...,4,046 단상 건축과 도시 파일럿 도시는 건축의 확장형이다 적어도 도시는 그 시대의 ...,"[(046 단상 건축과 도시 파일럿 도시는 건축의 확장형이다, 0, 31, None..."
...,...,...,...,...,...,...,...
13348,https://blog.naver.com/bubi-/222636081111,신해철 와이프 윤원희 똑 닮은 자녀 + 죽음 이유 집도의 강세훈...,2022.02.01.,31일 밤 방송된 KBS 2TV 자본주의학교에는\n도(故) 신해철 와이프 윤원희 씨...,13348,31일 밤 방송된 KBS 2TV 자본주의학교에는 도 신해철 와이프 윤원희 씨 그리고...,[(31일 밤 방송된 KBS 2TV 자본주의학교에는 도 신해철 와이프 윤원희 씨 그...
13349,https://blog.naver.com/talgodang/222635787110,설날에 심심풀이 땅콩으로 보는 '2022 임인년' 올해의 운세,2022.02.01.,2022 임인년 설날입니다.\n명절 연휴 다들 잘 보내고 계신지요? 저는 차례 드릴...,13349,2022 임인년 설날입니다 명절 연휴 다들 잘 보내고 계신지요 저는 차례 드릴 준비...,[(2022 임인년 설날입니다 명절 연휴 다들 잘 보내고 계신지요 저는 차례 드릴 ...
13350,https://blog.naver.com/imbee1229/222636077004,신세계백화점 타임스퀘어점 문화센터 꽃꽂이!,2022.02.01.,신세계백화점 타임스퀘어점\n문화센터 꽃꽂이 수강 후기!\n50m\n© NAVER C...,13350,신세계백화점 타임스퀘어점 문화센터 꽃꽂이 수강 후기 50m NAVER Corp 신세...,[(신세계백화점 타임스퀘어점 문화센터 꽃꽂이 수강 후기 50m NAVER Corp ...
13351,https://blog.naver.com/kji206/222635993490,"전북 가볼만한곳 고창 고인돌유적지,운곡습지,고창읍성 등",2022.02.01.,"전북 가볼만한곳 고창 고인돌유적지,운곡습지,고창읍성 등\n고창은 서해바다 여행도 할...",13351,전북 가볼만한곳 고창 고인돌유적지 운곡습지 고창읍성 등 고창은 서해바다 여행도 할 ...,[(전북 가볼만한곳 고창 고인돌유적지 운곡습지 고창읍성 등 고창은 서해바다 여행도 ...


In [11]:
# 데이터프레임을 10개 단위로 나누기
chunk_size = 50
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [8]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_2월_전처리.csv", index=False, encoding='UTF-8')

In [12]:
import pandas as pd
df2 = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_2월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx
0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,0
1,우웅 기준 오빠 목성의 위성 에우로파에는 관심없지만 이런 사랑이라면 훌훌 떠나도 좋...,0
2,우리반 임시 시간표인데 월요일 1교시 문학 임시라니까 언제까지 이러는 건지는 모르겠...,0
3,약간 학교를 가고 싶기도 안 가고 싶기도 함,0
4,물론 내 의사와는 상관없겠지만 껄껄 내 사랑 초코나무숲이 판매 종료 한다는 소식에 ...,0
...,...,...
510374,추천 메뉴로는 와인 페어링이 포함된 그랑 투어 Gran Tour 를 추천드립니다,9849
510375,호주 이탈리아 프랑스 와인과 어울리는 호화로운 5코스 세트 메뉴로 충분한 시간을 가...,9849
510376,메뉴 및 예약에 관한 자세한 내용은 공식 홈페이지를 참고하세요,9849
510377,https www florentino com au florentino Florent...,9849


In [13]:
for a in range(198,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_2월_전처리.csv", index=False, encoding='UTF-8')
df2

100%|██████████| 3/3 [00:08<00:00,  2.80s/it]


,split_str,org_idx
0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,0
1,우웅 기준 오빠 목성의 위성 에우로파에는 관심없지만 이런 사랑이라면 훌훌 떠나도 좋...,0
2,우리반 임시 시간표인데 월요일 1교시 문학 임시라니까 언제까지 이러는 건지는 모르겠...,0
3,약간 학교를 가고 싶기도 안 가고 싶기도 함,0
4,물론 내 의사와는 상관없겠지만 껄껄 내 사랑 초코나무숲이 판매 종료 한다는 소식에 ...,0
...,...,...
700401,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...,13352
700402,아동문학가이자 동요작곡가 그리고 아동문화운동가셨던 윤극영님은 푸른 하늘 은하수 하얀...,13352
700403,일제강점기 중 우리나라 가사를 붙인 노래들을 작곡하여 어린이들에게 민족에 대한 자긍...,13352
700404,새해 복 많이 받으세요,13352


In [23]:
a

198

In [24]:
df2 = df2[df2['org_idx'] < 9850]
df2

,split_str,org_idx
0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,0
1,우웅 기준 오빠 목성의 위성 에우로파에는 관심없지만 이런 사랑이라면 훌훌 떠나도 좋...,0
2,우리반 임시 시간표인데 월요일 1교시 문학 임시라니까 언제까지 이러는 건지는 모르겠...,0
3,약간 학교를 가고 싶기도 안 가고 싶기도 함,0
4,물론 내 의사와는 상관없겠지만 껄껄 내 사랑 초코나무숲이 판매 종료 한다는 소식에 ...,0
...,...,...
510374,추천 메뉴로는 와인 페어링이 포함된 그랑 투어 Gran Tour 를 추천드립니다,9849
510375,호주 이탈리아 프랑스 와인과 어울리는 호화로운 5코스 세트 메뉴로 충분한 시간을 가...,9849
510376,메뉴 및 예약에 관한 자세한 내용은 공식 홈페이지를 참고하세요,9849
510377,https www florentino com au florentino Florent...,9849


In [14]:
df2.isnull().sum()

split_str    0
org_idx      0
dtype: int64

In [15]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_2월_전처리.csv", index=False, encoding='UTF-8')